# Documentation for delta-deltapsi

## What are we doing?

We will use majiq's deltapsi pipeline as an example of the type of Bayesian-based modeling and analysis that could potentially be done to analyze changes in alternative splicing. The aim is to look at changes in alternative splicing percent spliced in (psi) ratios across the isoforms of a given gene alone relatively independent of changes in whole-gene expression.

Briefly, for each of 4 experimental conditions (Ime4 HS, Ime4 control, Mcherry HS, Mcherry control) we run deltapsi analysis on m6A pulled down fraction versus the input to get an idea of the splice junctions in local splice variation (LSV) units which are enriched in the m6A pulled down fraction relative to input (LSVs, see original MAJIQ paper). We hypothesize that a new measure "delta-deltapsi", which is derived by simply taking the difference between E(deltapsi) values from majiq of two different comparisons, Ime4 HS vs Ime4 control and Mcherry HS vs Mcherry control, will be representative of differences in m6A-enriched splicing between the conditions.

Prior to processing, we run samtools index to index each bam file and generate .bam.bai files, as required by majiq.

Then, we run majiq build with --simplify flag to remove technical/biological "noise" to simplify our interpretation. Gff3 annotation is downloaded from ensembl for this purpose (ftp://ftp.ensembl.org/pub/release-98/gff3/drosophila_melanogaster). Then, we run majiq deltapsi on each of the 4 conditions as described above. Then, we run majiq voila with --show-all flag to show E(deltapsi) computations across all junctions.

Finally, one last custom script extracts E(deltapsi) from the output tsv files and subtracts E(deltapsi) for HS vs control for both Ime4 and Mcherry. This allows us to order the expected delta-deltapsi values from greatest to least.

I have created a couple scripts which were run on the CHOP respublica cluster as found in the folder accompanying this notebook.

# References

https://elifesciences.org/articles/11752

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5704921/